<a href="https://colab.research.google.com/github/xcellentbird/Machine_Learning/blob/main/Deep_Learning_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PyTorch: Tensors

In [9]:
import torch

In [10]:
device = torch.device('cuda:0')

N, D_in, H, D_out = 64, 1000, 100, 10
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)
w1 = torch.randn(D_in, H, device=device)
w2 = torch.randn(H, D_out, device=device)

lr = 1e-6
epochs = 500

In [11]:
for t in range(epochs):
  h = x.mm(w1) # layer1. 행렬곱. size = 64 * 100.
  h_relu = h.clamp(min=0) # relu. clamp:최대 또는 최소값으로 깎는 함수. size = 64 * 100.
  y_pred = h_relu.mm(w2) # layer2. 행렬곱. size = 64 * 10
  loss = (y_pred - y).pow(2).sum() # loss 계산. (예측값 - 타겟값)^2의 합

  grad_y_pred = 2.0 * (y_pred - y) # loss 미분값. 2 * (예측값 - 타겟값). size = 64 * 10
  grad_w2 = h_relu.t().mm(grad_y_pred) # [100 * 64] * [64 * 10] = 100 * 10
  
  grad_h_relu = grad_y_pred.mm(w2.t()) # [64 * 10] * [10 * 100] = 64 * 100
  grad_h = grad_h_relu.clone()
  grad_h[h < 0] = 0 # h 요소 중에서 음수값에 해당하는 index를 가져와 grad_h에서 해당 index의 값을 0으로 설정한다.
  grad_w1 = x.t().mm(grad_h) # [1000 * 64] * [64 * 100] = 1000 * 100

  w1 -= lr * grad_w1
  w2 -= lr * grad_w2

## PyTorch: Autograd

In [18]:
N, D_in, H, D_out = 64, 1000, 100, 10
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)
w1 = torch.randn(D_in, H, device=device, requires_grad=True) # 학습이 필요한 가중치이므로 requires_grad= Ture 설정하여 미분 그래프를 그리도록 유도
w2 = torch.randn(H, D_out, device=device, requires_grad=True) # ...

lr = 1e-6
epochs = 500

In [19]:
for t in range(epochs):
  # forward(). 중간 값들을 수동으로 계산할 필요가 없다. 
  y_pred = x.mm(w1).clamp(min=0).mm(w2)
  loss = (y_pred - y).pow(2).sum()

  # backward()를 통해 미분값(grad)을 계산한다.
  loss.backward()

  # gradient를 추적할 필요없다는 것을 명시해두고, 가중치를 update한다.
  with torch.no_grad():
    w1 -= lr * w1.grad
    w2 -= lr * w2.grad
    # 가중치를 update했으니 다음 학습을 위해 grad를 0으로 초기화 시켜준다.
    w1.grad.zero_()
    w2.grad.zero_()

## PyTorch: New Autograd Functions

In [20]:
class MyReLU(torch.autograd.Function):
  @staticmethod
  def forward(ctx, x):
    ctx.save_for_backward(x)
    return x.clamp(min=0)

  @staticmethod
  def backward(ctx, grad_y):
    x, = ctx.saved_tensors
    grad_input = grad_y.clone()
    grad_input[x < 0] = 0
    return grad_input

In [ ]:
def my_relu(x):
  return MyReLU.apply(x)

## PyTorch: nn

In [24]:
device = torch.device('cuda:0')

N, D_in, H, D_out = 64, 1000, 100, 10
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out))

model.to(device)

Sequential(
  (0): Linear(in_features=1000, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=10, bias=True)
)

In [26]:
lr = 1e-6
epochs = 500
for t in range(epochs):
  y_pred = model(x)
  loss = torch.nn.functional.mse_loss(y_pred, y)

  loss.backward()

  with torch.no_grad():
    for param in model.parameters():
      param -= lr * param.grad
  model.zero_grad()

## PyTorch: optim

In [27]:
device = torch.device('cuda:0')

N, D_in, H, D_out = 64, 1000, 100, 10
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out))

model.to(device)

lr = 1e-6
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [28]:
epochs = 500
for t in range(epochs):
  y_pred = model(x)
  loss = torch.nn.functional.mse_loss(y_pred, y)

  loss.backward()

  optimizer.step()
  optimizer.zero_grad()

## PyTorch: nn Define new Modules

Sequential은 단순히 층층이 쌓는 구조였다면, Module은 좀 더 복잡하게 모델을 구성할 수 있게 한다.

In [41]:
class CustomNet(torch.nn.Module):
  def __init__(self, D_in, H, D_out):
    super(CustomNet, self).__init__()
    self.linear1 = torch.nn.Linear(D_in, H)
    self.linear2 = torch.nn.Linear(H, D_out)

  def forward(self, x):
    h_relu = self.linear1(x).clamp(min=0)
    y_pred = self.linear2(h_relu)
    return y_pred

In [42]:
device = torch.device('cuda:0')

N, D_in, H, D_out = 64, 1000, 100, 10
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)

model = CustomNet(D_in, H, D_out)
model.to(device)

lr = 1e-6
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [43]:
model

CustomNet(
  (linear1): Linear(in_features=1000, out_features=100, bias=True)
  (linear2): Linear(in_features=100, out_features=10, bias=True)
)

In [44]:
epochs = 500
for t in range(epochs):
  y_pred = model(x)
  loss = torch.nn.functional.mse_loss(y_pred, y)

  loss.backward()
  optimizer.step()
  optimizer.zero_grad()

## PyTorch: DataLoaders
dataloader를 사용하여 minibatch, shuffling, multithreading 등 다양한 방법으로 데이터를 가져올 수 있다.

In [45]:
from torch.utils.data import TensorDataset, DataLoader

device = torch.device('cuda:0')

N, D_in, H, D_out = 64, 1000, 100, 10
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)

In [46]:
loader = DataLoader(TensorDataset(x, y), batch_size=8)
model = CustomNet(D_in, H, D_out)
model.to(device)

lr = 1e-6
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [47]:
epochs = 20
for epoch in range(20):
  for x_batch, y_batch in loader:
    y_pred = model(x_batch)
    loss = torch.nn.functional.mse_loss(y_pred, y_batch)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

## PyTorch: Dynamic Computation Graphs
: 연산과 그래프를 쌓는 과정을 동시에 진행시키는 것을 Dynamic Computation Graph라고 합니다. 연산이 끝나고 그래프를 모두 없애기 때문에 비효율적일 수 있습니다.

## PyTorch: Static Computation Graphs
: Dynamic과 다르게 그래프를 쌓고 연산을 진행합니다. 연산이 끝나고 그래프를 없애지 않고 남겨두어 다음 eopch 때 재사용합니다.